In [1]:
import pandas as pd
import numpy as np

from PIL import Image 
import os
import cv2

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
root_path = 'dataset/color600'

In [3]:
# Loading the information about the images

data = [] # data placeholder for the dataframe
dataset = [] # filled later with color, grayscale or segmented


# now we walk down the entire file tree
for root, dirs, files in os.walk(root_path):

    #print("record: root=",root," dir=",dirs," files=",files)

    if dirs: 
    
        # we must be in one of the dataset-subfolders (/dataset, /dataset/color, /dataset/grayscale or /dataset/segmented)
  
        # extract the dataset type from the path
        if root.endswith(('color', 'grayscale', 'segmented')):
            dataset = root.split('/')[-1]

    if not dirs and files: 
    
        # we must be in one of the leaf folders containing the images

        # extract the image metadata from path
        subfolder_name = root.split('/')[-1]
        plant_parts = subfolder_name.split('___')
        species = plant_parts[0]
        disease = plant_parts[1]

        # filter down to only JPG, JPEG and PNG
        image_files = [i for i in files if i.lower().endswith(('jpg', 'jpeg', 'png'))]

        # for each image file generate one new dataframe row
        for filename in image_files:
            data.append([filename, root, dataset, species, disease])

# create dataframe from data array
df = pd.DataFrame(data, columns =['filename', 'path', 'dataset', 'species', 'disease'])

In [4]:
# Showing the classes of the data
df.groupby(by=["species", "disease"])['filename'].count()

species                           disease                             
color600\Apple                    Apple_scab                              600
                                  Black_rot                               600
                                  Cedar_apple_rust                        600
                                  healthy                                 600
color600\Cherry_(including_sour)  Powdery_mildew                          600
                                  healthy                                 600
color600\Corn_(maize)             Cercospora_leaf_spot Gray_leaf_spot     600
                                  Common_rust_                            600
                                  Northern_Leaf_Blight                    600
                                  healthy                                 600
color600\Grape                    Black_rot                               600
                                  Esca_(Black_Measles)                 

In [5]:
# Storing the images path
image_path = 'dataset/color600'

In [6]:
# convert the images and save them in a array
# converts the images to grayscale and resizes them to (28,28,1) for LeNet model
# without regularization

X = [] # stores the image 
y = [] # stores the plants class
diseases = [] #stores the diseases of the plants

for folder_name, _,filenames in os.walk(image_path):
    if folder_name !="dataset/color" and '.DS_Store' not in filenames:
        for file in filenames:
            if file != 'desktop.ini':
                file_path = folder_name +"/"+ file
                image = Image.open(file_path)
                image = image.convert('L') # converts images to grayscale
                image = np.array(image.resize((28,28))) #resizes images to (28,28)
                X.append(image)
                y.append(os.path.basename(folder_name.split('\\')[1].split('___')[0]))
                diseases.append(os.path.basename(folder_name.split('\\')[1]))
            else:
                pass
    else:
        pass
    
X = np.array(X)
y = np.array(y)

# Encoding the labels
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_le = le.fit_transform(y)

# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y_le, test_size=0.2, random_state=42)


In [7]:
# Model LeNet for plant classification without regularization

from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf

model_le1 = Sequential()

model_le1.add(Conv2D(filters = 126,
                     input_shape = (28,28,1),
                     kernel_size = (5,5),
                     activation = 'relu'))

model_le1.add(MaxPooling2D(pool_size = (2, 2)))

model_le1.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

model_le1.add(MaxPooling2D(pool_size = (2, 2)))

model_le1.add(Dropout(rate = 0.2))

model_le1.add(Flatten())

model_le1.add(Dense(units = 128,
                    activation = 'relu'))

model_le1.add(Dense(units = 9,
                     activation='softmax'))

model_le1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_le1_history = model_le1.fit(X_train, y_train, batch_size=200, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
80/80 [==============================] - 9s 106ms/step - loss: 3.5769 - accuracy: 0.3091 - val_loss: 1.6242 - val_accuracy: 0.4475
Epoch 2/100
80/80 [==============================] - 8s 105ms/step - loss: 1.5527 - accuracy: 0.4679 - val_loss: 1.3592 - val_accuracy: 0.5242
Epoch 3/100
80/80 [==============================] - 8s 100ms/step - loss: 1.3629 - accuracy: 0.5271 - val_loss: 1.2990 - val_accuracy: 0.5598
Epoch 4/100
80/80 [==============================] - 8s 105ms/step - loss: 1.2407 - accuracy: 0.5719 - val_loss: 1.1002 - val_accuracy: 0.6217
Epoch 5/100
80/80 [==============================] - 8s 105ms/step - loss: 1.1426 - accuracy: 0.6026 - val_loss: 1.1290 - val_accuracy: 0.6076
Epoch 6/100
80/80 [==============================] - 8s 105ms/step - loss: 1.0903 - accuracy: 0.6227 - val_loss: 1.0597 - val_accuracy: 0.6399
Epoch 7/100
80/80 [==============================] - 8s 104ms/step - loss: 1.0347 - accuracy: 0.6419 - val_loss: 0.9788 - val_accuracy: 0.6674

Epoch 58/100
80/80 [==============================] - 8s 101ms/step - loss: 0.1514 - accuracy: 0.9461 - val_loss: 0.8125 - val_accuracy: 0.8124
Epoch 59/100
80/80 [==============================] - 8s 103ms/step - loss: 0.1607 - accuracy: 0.9475 - val_loss: 0.8514 - val_accuracy: 0.7960
Epoch 60/100
80/80 [==============================] - 8s 100ms/step - loss: 0.1680 - accuracy: 0.9413 - val_loss: 0.8785 - val_accuracy: 0.7982
Epoch 61/100
80/80 [==============================] - 8s 102ms/step - loss: 0.1422 - accuracy: 0.9492 - val_loss: 0.8797 - val_accuracy: 0.8066
Epoch 62/100
80/80 [==============================] - 8s 100ms/step - loss: 0.1519 - accuracy: 0.9482 - val_loss: 0.9048 - val_accuracy: 0.7949
Epoch 63/100
80/80 [==============================] - 8s 103ms/step - loss: 0.1483 - accuracy: 0.9475 - val_loss: 0.8973 - val_accuracy: 0.8013
Epoch 64/100
80/80 [==============================] - 8s 101ms/step - loss: 0.1650 - accuracy: 0.9420 - val_loss: 0.9272 - val_accuracy:

In [8]:
#LeNet no regularization
# predicting the plats of the test set
y_pred_model_le1 = model_le1.predict(X_test)


# getting the Plant with highest probabilty 
y_pred = np.argmax(y_pred_model_le1, axis=1)

# classification report
print(classification_report(y_test, y_pred))

# confusion matrix 
pd.crosstab(y_test, y_pred, rownames=['Real'], colnames=['Predicted'])

124/124 [==============================] - 1s 10ms/step
              precision    recall  f1-score   support

           0       0.64      0.67      0.66       449
           1       0.85      0.72      0.78       258
           2       0.92      0.94      0.93       486
           3       0.93      0.95      0.94       503
           4       0.82      0.74      0.78       253
           5       0.72      0.69      0.70       252
           6       0.67      0.71      0.69       363
           7       0.82      0.56      0.67       235
           8       0.81      0.87      0.84      1161

    accuracy                           0.80      3960
   macro avg       0.80      0.76      0.78      3960
weighted avg       0.81      0.80      0.80      3960



Predicted,0,1,2,3,4,5,6,7,8
Real,,,,,,,,,
0,301,16,7,6,9,11,41,2,56
1,20,187,6,3,4,1,11,1,25
2,13,1,455,0,2,1,1,0,13
3,5,0,1,476,0,2,2,0,17
4,11,2,7,1,187,1,5,0,39
5,14,5,1,1,4,173,23,1,30
6,26,6,0,1,2,18,259,19,32
7,25,1,5,10,2,6,29,132,25
8,53,3,10,12,18,27,17,6,1015


In [9]:
# convert the images and save them in a array
# converts the images to grayscale and resizes them to (28,28,1) for LeNet model
# MinMax binary regularization

X = [] # stores the image 
y = [] # stores the plants class
diseases = [] #stores the diseases of the plants

for folder_name, _,filenames in os.walk(image_path):
    if folder_name !="dataset/color" and '.DS_Store' not in filenames:
        for file in filenames:
            if file != 'desktop.ini':
                file_path = folder_name +"/"+ file
                image = Image.open(file_path)
                image = image.convert('L') # converts images to grayscale
                image = np.array(image.resize((28,28))) #resizes images to (28,28)
                ret, thresh = cv2.threshold(image,140,255,cv2.THRESH_BINARY)
                image = cv2.normalize(thresh, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                X.append(image)
                y.append(os.path.basename(folder_name.split('\\')[1].split('___')[0]))
                diseases.append(os.path.basename(folder_name.split('\\')[1]))
            else:
                pass
    else:
        pass
    
X = np.array(X)
y = np.array(y)

# Encoding the labels
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_le = le.fit_transform(y)

# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y_le, test_size=0.2, random_state=42)


In [10]:
# Model LeNet for plant classification with MinMax norm with binary output regularization

from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf

model_le2 = Sequential()

model_le2.add(Conv2D(filters = 126,
                     input_shape = (28,28,1),
                     kernel_size = (5,5),
                     activation = 'relu'))

model_le2.add(MaxPooling2D(pool_size = (2, 2)))

model_le2.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

model_le2.add(MaxPooling2D(pool_size = (2, 2)))

model_le2.add(Dropout(rate = 0.2))

model_le2.add(Flatten())

model_le2.add(Dense(units = 128,
                    activation = 'relu'))

model_le2.add(Dense(units = 9,
                     activation='softmax'))

model_le2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_le2_history = model_le2.fit(X_train, y_train, batch_size=200, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
80/80 [==============================] - 9s 103ms/step - loss: 1.6416 - accuracy: 0.4443 - val_loss: 1.4374 - val_accuracy: 0.5053
Epoch 2/100
80/80 [==============================] - 8s 101ms/step - loss: 1.2947 - accuracy: 0.5574 - val_loss: 1.2138 - val_accuracy: 0.5813
Epoch 3/100
80/80 [==============================] - 8s 102ms/step - loss: 1.1500 - accuracy: 0.5996 - val_loss: 1.1589 - val_accuracy: 0.6015
Epoch 4/100
80/80 [==============================] - 8s 102ms/step - loss: 1.0542 - accuracy: 0.6374 - val_loss: 1.0899 - val_accuracy: 0.6187
Epoch 5/100
80/80 [==============================] - 8s 101ms/step - loss: 0.9760 - accuracy: 0.6607 - val_loss: 1.0443 - val_accuracy: 0.6407
Epoch 6/100
80/80 [==============================] - 8s 100ms/step - loss: 0.9183 - accuracy: 0.6779 - val_loss: 1.0268 - val_accuracy: 0.6419
Epoch 7/100
80/80 [==============================] - 8s 102ms/step - loss: 0.8583 - accuracy: 0.7001 - val_loss: 1.0002 - val_accuracy: 0.6551

Epoch 58/100
80/80 [==============================] - 8s 101ms/step - loss: 0.1308 - accuracy: 0.9559 - val_loss: 1.9108 - val_accuracy: 0.6760
Epoch 59/100
80/80 [==============================] - 8s 101ms/step - loss: 0.1317 - accuracy: 0.9554 - val_loss: 1.9136 - val_accuracy: 0.6811
Epoch 60/100
80/80 [==============================] - 8s 101ms/step - loss: 0.1320 - accuracy: 0.9534 - val_loss: 1.8677 - val_accuracy: 0.6816
Epoch 61/100
80/80 [==============================] - 8s 102ms/step - loss: 0.1237 - accuracy: 0.9583 - val_loss: 1.9476 - val_accuracy: 0.6763
Epoch 62/100
80/80 [==============================] - 8s 101ms/step - loss: 0.1224 - accuracy: 0.9577 - val_loss: 1.9613 - val_accuracy: 0.6846
Epoch 63/100
80/80 [==============================] - 8s 104ms/step - loss: 0.1217 - accuracy: 0.9598 - val_loss: 1.9375 - val_accuracy: 0.6715
Epoch 64/100
80/80 [==============================] - 9s 107ms/step - loss: 0.1235 - accuracy: 0.9588 - val_loss: 1.9226 - val_accuracy:

In [11]:
# LeNet MinMax binary regularization
# predicting the the plants of the test set
y_pred_model_le2 = model_le2.predict(X_test)

# getting the Plant with highest probabilty 
y_pred = np.argmax(y_pred_model_le2, axis=1)

# classification report
print(classification_report(y_test, y_pred))

# confusion matrix 
pd.crosstab(y_test, y_pred, rownames=['Real'], colnames=['Predicted'])

124/124 [==============================] - 1s 9ms/step
              precision    recall  f1-score   support

           0       0.47      0.43      0.45       449
           1       0.64      0.56      0.60       258
           2       0.81      0.85      0.83       486
           3       0.84      0.84      0.84       503
           4       0.58      0.56      0.57       253
           5       0.64      0.62      0.63       252
           6       0.57      0.44      0.50       363
           7       0.63      0.52      0.57       235
           8       0.68      0.78      0.73      1161

    accuracy                           0.67      3960
   macro avg       0.65      0.62      0.63      3960
weighted avg       0.67      0.67      0.67      3960



Predicted,0,1,2,3,4,5,6,7,8
Real,,,,,,,,,
0,192,15,23,19,22,13,33,9,123
1,25,144,7,2,15,7,12,4,42
2,12,7,414,2,11,1,0,2,37
3,14,0,1,425,3,7,7,9,37
4,23,6,13,1,142,4,1,3,60
5,20,6,7,6,3,157,20,1,32
6,43,15,3,10,5,32,161,32,62
7,16,7,5,12,3,7,26,122,37
8,67,24,41,30,40,18,21,11,909


In [12]:
# convert the images and save them in a array
# converts the images to grayscale and resizes them to (28,28,1) for LeNet model
# MinMax regularization

X = [] # stores the image 
y = [] # stores the plants class
diseases = [] #stores the diseases of the plants

for folder_name, _,filenames in os.walk(image_path):
    if folder_name !="dataset/color" and '.DS_Store' not in filenames:
        for file in filenames:
            if file != 'desktop.ini':
                file_path = folder_name +"/"+ file
                image = Image.open(file_path)
                image = image.convert('L') # converts images to grayscale
                image = np.array(image.resize((28,28))) #resizes images to (28,28)
                image= cv2.normalize(image, None, 0, 1.0,
                                    cv2.NORM_MINMAX, dtype=cv2.CV_32F) # regularization
                X.append(image)
                y.append(os.path.basename(folder_name.split('\\')[1].split('___')[0]))
                diseases.append(os.path.basename(folder_name.split('\\')[1]))
            else:
                pass
    else:
        pass
    
X = np.array(X)
y = np.array(y)

# Encoding the labels
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_le = le.fit_transform(y)

# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y_le, test_size=0.2, random_state=42)


In [13]:
# Model LeNet for plant classification with MinMax norm regularization

from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf

model_le3 = Sequential()

model_le3.add(Conv2D(filters = 126,
                     input_shape = (28,28,1),
                     kernel_size = (5,5),
                     activation = 'relu'))

model_le3.add(MaxPooling2D(pool_size = (2, 2)))

model_le3.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

model_le3.add(MaxPooling2D(pool_size = (2, 2)))

model_le3.add(Dropout(rate = 0.2))

model_le3.add(Flatten())

model_le3.add(Dense(units = 128,
                    activation = 'relu'))

model_le3.add(Dense(units = 9,
                     activation='softmax'))

model_le3.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_le3_history = model_le3.fit(X_train, y_train, batch_size=200, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
80/80 [==============================] - 9s 104ms/step - loss: 1.7051 - accuracy: 0.4133 - val_loss: 1.4184 - val_accuracy: 0.5119
Epoch 2/100
80/80 [==============================] - 8s 102ms/step - loss: 1.2365 - accuracy: 0.5753 - val_loss: 1.1519 - val_accuracy: 0.6114
Epoch 3/100
80/80 [==============================] - 8s 102ms/step - loss: 1.0474 - accuracy: 0.6415 - val_loss: 0.9914 - val_accuracy: 0.6646
Epoch 4/100
80/80 [==============================] - 8s 103ms/step - loss: 0.9336 - accuracy: 0.6819 - val_loss: 0.9126 - val_accuracy: 0.6944
Epoch 5/100
80/80 [==============================] - 8s 101ms/step - loss: 0.8309 - accuracy: 0.7142 - val_loss: 0.8108 - val_accuracy: 0.7237
Epoch 6/100
80/80 [==============================] - 8s 103ms/step - loss: 0.7609 - accuracy: 0.7404 - val_loss: 0.7541 - val_accuracy: 0.7462
Epoch 7/100
80/80 [==============================] - 8s 105ms/step - loss: 0.7113 - accuracy: 0.7592 - val_loss: 0.7444 - val_accuracy: 0.7482

Epoch 58/100
80/80 [==============================] - 8s 101ms/step - loss: 0.0771 - accuracy: 0.9731 - val_loss: 0.6351 - val_accuracy: 0.8460
Epoch 59/100
80/80 [==============================] - 8s 103ms/step - loss: 0.0632 - accuracy: 0.9808 - val_loss: 0.6359 - val_accuracy: 0.8465
Epoch 60/100
80/80 [==============================] - 8s 101ms/step - loss: 0.0603 - accuracy: 0.9802 - val_loss: 0.6245 - val_accuracy: 0.8510
Epoch 61/100
80/80 [==============================] - 8s 102ms/step - loss: 0.0635 - accuracy: 0.9782 - val_loss: 0.6286 - val_accuracy: 0.8439
Epoch 62/100
80/80 [==============================] - 8s 102ms/step - loss: 0.0564 - accuracy: 0.9816 - val_loss: 0.6298 - val_accuracy: 0.8495
Epoch 63/100
80/80 [==============================] - 8s 102ms/step - loss: 0.0561 - accuracy: 0.9802 - val_loss: 0.6571 - val_accuracy: 0.8462
Epoch 64/100
80/80 [==============================] - 8s 101ms/step - loss: 0.0548 - accuracy: 0.9814 - val_loss: 0.6686 - val_accuracy:

In [14]:
# LeNet MinMax regularization
# predicting the the plants of the test set
y_pred_model_le3 = model_le3.predict(X_test)

# getting the Plant with highest probabilty 
y_pred = np.argmax(y_pred_model_le3, axis=1)

# classification report
print(classification_report(y_test, y_pred))

# confusion matrix 
pd.crosstab(y_test, y_pred, rownames=['Real'], colnames=['Predicted'])

124/124 [==============================] - 1s 9ms/step
              precision    recall  f1-score   support

           0       0.68      0.82      0.74       449
           1       0.83      0.86      0.85       258
           2       0.96      0.93      0.94       486
           3       0.96      0.95      0.95       503
           4       0.80      0.85      0.82       253
           5       0.86      0.73      0.79       252
           6       0.76      0.73      0.75       363
           7       0.84      0.68      0.75       235
           8       0.88      0.88      0.88      1161

    accuracy                           0.85      3960
   macro avg       0.84      0.83      0.83      3960
weighted avg       0.85      0.85      0.85      3960



Predicted,0,1,2,3,4,5,6,7,8
Real,,,,,,,,,
0,370,14,5,0,11,1,15,5,28
1,13,221,2,0,2,1,7,1,11
2,16,6,451,0,6,1,0,0,6
3,6,4,0,477,1,0,2,2,11
4,16,2,3,0,214,2,0,0,16
5,11,3,1,0,2,185,30,1,19
6,34,7,0,1,5,11,266,12,27
7,20,3,2,8,3,2,15,160,22
8,61,5,5,10,23,11,15,9,1022


In [4]:
# AlexNet MinMax norm regularization
# import dataset as (227,227,3) with normalizing by dividing by 255
## rounding to 3 digits because of memory limitations

#convert the images and save them in a list. 
X = [] # stores the image 
y = [] # stores the plants class
diseases = [] #stores the diseases of the plants

for folder_name, _,filenames in os.walk(image_path):
    if folder_name !="dataset/color600" and '.DS_Store' not in filenames:
        for file in filenames:
            if file != 'desktop.ini':
                file_path = folder_name +"/"+ file
                image = Image.open(file_path)
                image = np.array(image.convert('RGB'))
                image = cv2.resize(image, (227,227))
                image= cv2.normalize(image, None, 0, 1.0,cv2.NORM_MINMAX, dtype=cv2.CV_32F) # regularization
                #image = image.resize((227,227)) # resizing the images to (227,227,3)
                #image = np.round(image/255, 3) # regularization
                #image= cv2.normalize(image, image, 0, 1.0,cv2.NORM_MINMAX, dtype=cv2.CV_32F) # regularization
                X.append(image)
                y.append(os.path.basename(folder_name.split('\\')[1].split('___')[0]))
                diseases.append(os.path.basename(folder_name.split('\\')[1]))
            else:
                pass
    else:
        pass

X = np.array(X).astype('float32')

# Encoding the labels
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

y_le = le.fit_transform(y)
y_le = np.array(y_le).astype('float32')

# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y_le, test_size=0.2, random_state=42)

In [6]:
#AlexNet for Plant identification with MinMax norm normalization

from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, MaxPool2D
import tensorflow as tf


model_alex1 = Sequential()

model_alex1.add(Conv2D(filters=96, kernel_size=(11, 11), 
                        strides=(4, 4), activation="relu", 
                        input_shape=(227, 227, 3)))
model_alex1.add(BatchNormalization())
model_alex1.add(MaxPool2D(pool_size=(3, 3), strides= (2, 2)))
model_alex1.add(Conv2D(filters=256, kernel_size=(5, 5), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model_alex1.add(BatchNormalization())
model_alex1.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
model_alex1.add(Conv2D(filters=384, kernel_size=(3, 3), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model_alex1.add(BatchNormalization())
model_alex1.add(Conv2D(filters=384, kernel_size=(3, 3), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model_alex1.add(BatchNormalization())
model_alex1.add(Conv2D(filters=256, kernel_size=(3, 3), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model_alex1.add(BatchNormalization())
model_alex1.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
model_alex1.add(Flatten())
model_alex1.add(Dense(4096, activation="relu"))
model_alex1.add(Dropout(0.5))
model_alex1.add(Dense(9, activation="softmax"))

# model_alex1.compile(loss='sparse_categorical_crossentropy', 
#               optimizer=tf.optimizers.SGD(lr=0.001), 
#               metrics=['accuracy'])

#model_alex1.summary()

model_alex1.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_alex1_history = model_alex1.fit(X_train, y_train, batch_size=64, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
248/248 [==============================] - 374s 2s/step - loss: 4.6338 - accuracy: 0.5420 - val_loss: 3.0731 - val_accuracy: 0.4422
Epoch 2/20
248/248 [==============================] - 366s 1s/step - loss: 0.7612 - accuracy: 0.7556 - val_loss: 1.1328 - val_accuracy: 0.6520
Epoch 3/20
248/248 [==============================] - 369s 1s/step - loss: 0.5536 - accuracy: 0.8204 - val_loss: 1.6246 - val_accuracy: 0.5876
Epoch 4/20
248/248 [==============================] - 369s 1s/step - loss: 0.4175 - accuracy: 0.8664 - val_loss: 1.0214 - val_accuracy: 0.7528
Epoch 5/20
248/248 [==============================] - 370s 1s/step - loss: 0.3466 - accuracy: 0.8896 - val_loss: 2.7238 - val_accuracy: 0.5982
Epoch 6/20
248/248 [==============================] - 367s 1s/step - loss: 0.3071 - accuracy: 0.9061 - val_loss: 1.4435 - val_accuracy: 0.6684
Epoch 7/20
248/248 [==============================] - 369s 1s/step - loss: 0.2885 - accuracy: 0.9136 - val_loss: 3.3770 - val_accuracy: 0.6053

In [7]:
# AlexNet MinMax norm regularization
# predicting the the plants of the test set
y_pred_model_alex1 = model_alex1.predict(X_test)

# getting the Plant with highest probabilty 
y_pred = np.argmax(y_pred_model_alex1, axis=1)

# classification report
print(classification_report(y_test, y_pred))

# confusion matrix 
pd.crosstab(y_test, y_pred, rownames=['Real'], colnames=['Predicted'])

124/124 [==============================] - 25s 198ms/step
              precision    recall  f1-score   support

         0.0       0.56      0.90      0.69       449
         1.0       0.83      0.43      0.57       258
         2.0       0.99      0.96      0.98       486
         3.0       1.00      0.87      0.93       503
         4.0       0.51      1.00      0.68       253
         5.0       0.42      0.98      0.59       252
         6.0       0.93      0.18      0.30       363
         7.0       0.99      0.71      0.83       235
         8.0       0.97      0.73      0.83      1161

    accuracy                           0.76      3960
   macro avg       0.80      0.75      0.71      3960
weighted avg       0.85      0.76      0.75      3960



Predicted,0,1,2,3,4,5,6,7,8
Real,,,,,,,,,
0.0,402,8,2,0,29,8,0,0,0
1.0,34,112,0,0,55,57,0,0,0
2.0,6,0,468,0,10,2,0,0,0
3.0,32,0,0,438,7,25,0,0,1
4.0,1,0,0,0,252,0,0,0,0
5.0,2,0,0,0,3,247,0,0,0
6.0,68,7,1,0,8,191,64,1,23
7.0,58,1,1,0,2,3,0,168,2
8.0,120,7,1,1,124,56,5,1,846


In [4]:
# import dataset as (227,227,3) for AlexNet model with MinMax binary normalization

#convert the images and save them in a list. 
X = [] # stores the image 
y = [] # stores the plants class
diseases = [] #stores the diseases of the plants

for folder_name, _,filenames in os.walk(image_path):
    if folder_name !="dataset/color" and '.DS_Store' not in filenames:
        for file in filenames:
            if file != 'desktop.ini':
                file_path = folder_name +"/"+ file
                image = Image.open(file_path)
                image = image.convert('RGB')
                image = np.array(image.resize((227,227)))
                ret, thresh = cv2.threshold(image,140,255,cv2.THRESH_BINARY)
                image = cv2.normalize(thresh, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                X.append(image)
                y.append(os.path.basename(folder_name.split('\\')[1].split('___')[0]))
                diseases.append(os.path.basename(folder_name.split('\\')[1]))
            else:
                pass
    else:
        pass

X = np.array(X).astype('float32')

# Encoding the labels
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

y_le = le.fit_transform(y)

y_le = np.array(y_le).astype('float32')
# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y_le, test_size=0.2, random_state=42)


In [6]:
# AlexNet for Plant identification with MinMax norm binary normalization
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, MaxPool2D
import tensorflow as tf


model_alex2 = Sequential()

model_alex2.add(Conv2D(filters=96, kernel_size=(11, 11), 
                        strides=(4, 4), activation="relu", 
                        input_shape=(227, 227, 3)))
model_alex2.add(BatchNormalization())
model_alex2.add(MaxPool2D(pool_size=(3, 3), strides= (2, 2)))
model_alex2.add(Conv2D(filters=256, kernel_size=(5, 5), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model_alex2.add(BatchNormalization())
model_alex2.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
model_alex2.add(Conv2D(filters=384, kernel_size=(3, 3), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model_alex2.add(BatchNormalization())
model_alex2.add(Conv2D(filters=384, kernel_size=(3, 3), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model_alex2.add(BatchNormalization())
model_alex2.add(Conv2D(filters=256, kernel_size=(3, 3), 
                        strides=(1, 1), activation="relu", 
                        padding="same"))
model_alex2.add(BatchNormalization())
model_alex2.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
model_alex2.add(Flatten())
model_alex2.add(Dense(4096, activation="relu"))
model_alex2.add(Dropout(0.5))
model_alex2.add(Dense(9, activation="softmax"))

# model_alex2.compile(loss='sparse_categorical_crossentropy', 
#               optimizer=tf.optimizers.SGD(lr=0.001), 
#               metrics=['accuracy'])

#model_alex2.summary()

model_alex2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_alex2_history = model_alex2.fit(X_train, y_train, batch_size=64, epochs=30, validation_data=(X_test, y_test))

Epoch 1/30
248/248 [==============================] - 351s 1s/step - loss: 6.1867 - accuracy: 0.4778 - val_loss: 2.1431 - val_accuracy: 0.5222
Epoch 2/30
248/248 [==============================] - 339s 1s/step - loss: 1.0076 - accuracy: 0.6631 - val_loss: 1.1655 - val_accuracy: 0.6391
Epoch 3/30
248/248 [==============================] - 340s 1s/step - loss: 0.7819 - accuracy: 0.7331 - val_loss: 0.8958 - val_accuracy: 0.7078
Epoch 4/30
248/248 [==============================] - 339s 1s/step - loss: 0.6596 - accuracy: 0.7823 - val_loss: 0.6311 - val_accuracy: 0.7975
Epoch 5/30
248/248 [==============================] - 340s 1s/step - loss: 0.6065 - accuracy: 0.8015 - val_loss: 0.7494 - val_accuracy: 0.7682
Epoch 6/30
248/248 [==============================] - 339s 1s/step - loss: 0.5299 - accuracy: 0.8278 - val_loss: 0.4836 - val_accuracy: 0.8462
Epoch 7/30
248/248 [==============================] - 338s 1s/step - loss: 0.4735 - accuracy: 0.8507 - val_loss: 0.6538 - val_accuracy: 0.8023

In [7]:
# AlexNet with MinMax norm binary normalization
# predicting the the plants of the test set
y_pred_model_alex2 = model_alex2.predict(X_test)

# getting the Plant with highest probabilty 
y_pred = np.argmax(y_pred_model_alex2, axis=1)

# classification report
print(classification_report(y_test, y_pred))

# confusion matrix 
pd.crosstab(y_test, y_pred, rownames=['Real'], colnames=['Predicted'])

124/124 [==============================] - 24s 193ms/step
              precision    recall  f1-score   support

         0.0       0.82      0.90      0.86       449
         1.0       0.97      0.91      0.94       258
         2.0       0.98      0.96      0.97       486
         3.0       0.98      0.98      0.98       503
         4.0       0.85      0.92      0.88       253
         5.0       0.92      0.90      0.91       252
         6.0       0.95      0.79      0.86       363
         7.0       0.97      0.88      0.92       235
         8.0       0.90      0.94      0.92      1161

    accuracy                           0.92      3960
   macro avg       0.93      0.91      0.92      3960
weighted avg       0.92      0.92      0.92      3960



Predicted,0,1,2,3,4,5,6,7,8
Real,,,,,,,,,
0.0,404,1,2,0,11,4,3,4,20
1.0,6,236,1,1,0,3,0,0,11
2.0,9,0,466,0,3,0,0,0,8
3.0,1,0,0,493,0,0,0,1,8
4.0,6,0,0,0,232,0,0,0,15
5.0,5,0,0,0,8,226,1,1,11
6.0,20,1,1,2,3,7,287,0,42
7.0,10,0,6,4,1,0,1,206,7
8.0,32,5,0,5,14,6,9,1,1089
